In [ ]:
import os 
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

### LCEL 인터페이스
- 사용자 정의 체인을 가능한 쉽게 만들 수 있게 구현한 Runnable 프로토콜
- stream : 응답 청크 스트리밍
- invoke : 입력에 대한 체인 호출
- batch : 입력 목록에 대해 체인 호출

In [3]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

model = ChatOpenAI(api_key=OPENAI_API_KEY)
prompt = ChatPromptTemplate.from_template("{topic}에 대하여 3문장으로 설명해줘")
chain = prompt | model

In [26]:
chain.input_schema.schema()

# 여기선 체인의 입력 스키마는 첫 번째 부분인 프롬프트의 입력 스키마다.  

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}

In [27]:
chain.input_schema.schema_json()

'{"title": "PromptInput", "type": "object", "properties": {"topic": {"title": "Topic", "type": "string"}}}'

In [28]:
import json 

def print_schema(schema):
    print(json.dumps(schema, indent=4))
    
input_schema = {
    "type": "object",
    "properties": {
        "name" : {"type" : "string"},
        "age" : {"type" : "integer", "minimum" : 0},
    },
    "required" : ["name", "age"]
}

print_schema(input_schema)

{
    "type": "object",
    "properties": {
        "name": {
            "type": "string"
        },
        "age": {
            "type": "integer",
            "minimum": 0
        }
    },
    "required": [
        "name",
        "age"
    ]
}


In [29]:
model.input_schema.schema()

{'title': 'ChatOpenAIInput',
 'anyOf': [{'type': 'string'},
  {'$ref': '#/definitions/StringPromptValue'},
  {'$ref': '#/definitions/ChatPromptValueConcrete'},
  {'type': 'array',
   'items': {'anyOf': [{'$ref': '#/definitions/AIMessage'},
     {'$ref': '#/definitions/HumanMessage'},
     {'$ref': '#/definitions/ChatMessage'},
     {'$ref': '#/definitions/SystemMessage'},
     {'$ref': '#/definitions/FunctionMessage'},
     {'$ref': '#/definitions/ToolMessage'}]}}],
 'definitions': {'StringPromptValue': {'title': 'StringPromptValue',
   'description': 'String prompt value.',
   'type': 'object',
   'properties': {'text': {'title': 'Text', 'type': 'string'},
    'type': {'title': 'Type',
     'default': 'StringPromptValue',
     'enum': ['StringPromptValue'],
     'type': 'string'}},
   'required': ['text']},
  'ToolCall': {'title': 'ToolCall',
   'type': 'object',
   'properties': {'name': {'title': 'Name', 'type': 'string'},
    'args': {'title': 'Args', 'type': 'object'},
    'id': {

In [30]:
chain.output_schema.schema()

{'title': 'ChatOpenAIOutput',
 'anyOf': [{'$ref': '#/definitions/AIMessage'},
  {'$ref': '#/definitions/HumanMessage'},
  {'$ref': '#/definitions/ChatMessage'},
  {'$ref': '#/definitions/SystemMessage'},
  {'$ref': '#/definitions/FunctionMessage'},
  {'$ref': '#/definitions/ToolMessage'}],
 'definitions': {'ToolCall': {'title': 'ToolCall',
   'type': 'object',
   'properties': {'name': {'title': 'Name', 'type': 'string'},
    'args': {'title': 'Args', 'type': 'object'},
    'id': {'title': 'Id', 'type': 'string'}},
   'required': ['name', 'args', 'id']},
  'InvalidToolCall': {'title': 'InvalidToolCall',
   'type': 'object',
   'properties': {'name': {'title': 'Name', 'type': 'string'},
    'args': {'title': 'Args', 'type': 'string'},
    'id': {'title': 'Id', 'type': 'string'},
    'error': {'title': 'Error', 'type': 'string'}},
   'required': ['name', 'args', 'id', 'error']},
  'UsageMetadata': {'title': 'UsageMetadata',
   'type': 'object',
   'properties': {'input_tokens': {'title':

In [4]:
# chain.stream 메서드를 사용하여 '멀티모달' 토픽에 대한 스트림을 생성하고 반복합니다.
for s in chain.stream({"topic": "멀티모달"}):
    # 스트림에서 받은 데이터의 내용을 출력합니다. 줄바꿈 없이 이어서 출력하고, 버퍼를 즉시 비웁니다.
    print(s.content, end="", flush=True)

멀티모달은 여러 가지 형태의 정보를 결합하여 전달하는 것을 말합니다. 이는 텍스트, 이미지, 음성, 동영상 등 다양한 매체를 활용하여 사용자에게 정보를 제공하는 방식입니다. 멀티모달은 사용자 경험을 향상시키고 정보 전달을 보다 효과적으로 할 수 있는 방법 중 하나입니다.

In [7]:
chain.invoke({"topic" : "ChatGPT"})

AIMessage(content='ChatGPT는 자연어 처리 기술을 이용하여 대화를 수행하는 인공지능 챗봇 서비스입니다. 사용자의 질문에 응답하고 대화를 이어가며 다양한 주제에 대해 대화할 수 있습니다. ChatGPT는 학습을 통해 계속 발전하고 사용자와의 상호작용을 통해 더욱 더 유용한 서비스를 제공합니다.', response_metadata={'token_usage': {'completion_tokens': 126, 'prompt_tokens': 23, 'total_tokens': 149}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-7056738a-bcaf-415c-b4f6-30441ef74f55-0', usage_metadata={'input_tokens': 23, 'output_tokens': 126, 'total_tokens': 149})

In [8]:
# 비동기 스트림
async for s in chain.astream({"topic" : "Youtube"}):
    print(s.content, end="", flush=True)

Youtube는 구글이 소유하고 있는 동영상 공유 플랫폼으로 사용자들이 영상을 업로드하고 시청할 수 있는 서비스이다. 다양한 콘텐츠를 제공하며 라이브 방송도 가능하며, 광고 수익을 얻을 수 있는 유튜버들이 많이 활동하는 곳이다. 또한 다양한 장르의 영상을 검색하고 구독하여 자신의 관심사에 맞는 콘텐츠를 즐길 수 있다.

In [9]:
await chain.ainvoke({"topic": "NVDA"})

AIMessage(content='NVDA는 NVIDIA Corporation의 네임스페이스이며, 주로 NVIDIA의 주가를 나타내는 심볼로 사용된다. 주로 주식 시장에서 NVIDIA의 주가 동향을 살펴보는 데 사용된다. 또한, NVDA는 NVIDIA의 제품 및 기술과 관련된 뉴스와 정보를 나타내는 데에도 사용된다.', response_metadata={'token_usage': {'completion_tokens': 114, 'prompt_tokens': 22, 'total_tokens': 136}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-4a0e773c-0b44-4c4b-9beb-3fca86789c96-0', usage_metadata={'input_tokens': 22, 'output_tokens': 114, 'total_tokens': 136})

In [13]:
# JSONPatch 청크 스트리밍
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

vectorstore = FAISS.from_texts(
    ["테디가 살고 있는 곳은 대한민국 입니다."], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

retrieval_chain = (
    {
        "context": retriever.with_config(run_name="Docs"),
        "question": RunnablePassthrough(),
    }
    | prompt
    | model
    | StrOutputParser()
)

async for chunk in retrieval_chain.astream_log(
    "테디가 살고 있는 곳은 어딘가요?", include_names=["Docs"], diff=False
):
    print("-" * 40)
    print(chunk)

----------------------------------------
RunLog({'final_output': None,
 'id': 'd6357160-c1d7-448c-bdde-93cd33696010',
 'logs': {},
 'name': 'RunnableSequence',
 'streamed_output': [],
 'type': 'chain'})
----------------------------------------
RunLog({'final_output': None,
 'id': 'd6357160-c1d7-448c-bdde-93cd33696010',
 'logs': {'Docs': {'end_time': None,
                   'final_output': None,
                   'id': '68dee279-3014-492f-83d5-b82a77c78948',
                   'metadata': {},
                   'name': 'Docs',
                   'start_time': '2024-06-03T14:30:25.284+00:00',
                   'streamed_output': [],
                   'streamed_output_str': [],
                   'tags': ['map:key:context', 'FAISS', 'OpenAIEmbeddings'],
                   'type': 'retriever'}},
 'name': 'RunnableSequence',
 'streamed_output': [],
 'type': 'chain'})
----------------------------------------
RunLog({'final_output': None,
 'id': 'd6357160-c1d7-448c-bdde-93cd33696010',
 'l

In [14]:
# Parallel : 병렬성

from langchain_core.runnables import RunnableParallel

chain_1 = ChatPromptTemplate.from_template("{country}의 수도는 어디야?") | model 
chain_2 = ChatPromptTemplate.from_template("{country}의 면적을 얼마야?") | model 
combined = RunnableParallel(capital = chain_1, area = chain_2)

In [15]:
chain_1.invoke(
    {"country" : "대한민국"}
)

AIMessage(content='대한민국의 수도는 서울이다.', response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 22, 'total_tokens': 38}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-61a68291-5a54-4470-b5ef-5154ab716d3f-0', usage_metadata={'input_tokens': 22, 'output_tokens': 16, 'total_tokens': 38})

In [16]:
chain_2.invoke(
    {"country" : "대한민국"}
)

AIMessage(content='대한민국의 총 면적은 약 100,363 제곱 킬로미터입니다.', response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 24, 'total_tokens': 54}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-9fdf69e0-ddc5-44c3-9022-1e96d1e27b53-0', usage_metadata={'input_tokens': 24, 'output_tokens': 30, 'total_tokens': 54})

In [19]:
combined.invoke({"country" : "대한민국"})

{'capital': AIMessage(content='대한민국의 수도는 서울이다.', response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 22, 'total_tokens': 38}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-218c5042-ebc8-4321-98fe-436817d06b2b-0', usage_metadata={'input_tokens': 22, 'output_tokens': 16, 'total_tokens': 38}),
 'area': AIMessage(content='대한민국의 총 면적은 약 100,363km² 입니다.', response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 24, 'total_tokens': 49}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-66254aab-a26b-47b4-933a-d75ec4aa9667-0', usage_metadata={'input_tokens': 24, 'output_tokens': 25, 'total_tokens': 49})}

In [20]:
# 배치 병렬 처리 

chain_1.batch([{"country" : "대한민국"}, {"country" : "미국"}])

[AIMessage(content='대한민국의 수도는 서울이다.', response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 22, 'total_tokens': 38}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-99beae80-ccdb-4ae7-a6d0-379c6329790b-0', usage_metadata={'input_tokens': 22, 'output_tokens': 16, 'total_tokens': 38}),
 AIMessage(content='미국의 수도는 워싱턴 D.C.입니다.', response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 19, 'total_tokens': 36}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0d9db140-1c5b-4129-97f3-f0d15f9ff6f2-0', usage_metadata={'input_tokens': 19, 'output_tokens': 17, 'total_tokens': 36})]

In [21]:
chain_2.batch([{"country": "대한민국"}, {"country": "미국"}])

[AIMessage(content='대한민국의 면적은 약 100,363 km² 입니다.', response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 24, 'total_tokens': 46}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-81994e26-2f7c-442b-9722-d6cb33813954-0', usage_metadata={'input_tokens': 24, 'output_tokens': 22, 'total_tokens': 46}),
 AIMessage(content='미국의 총 면적은 약 9,826,675 km²이다.', response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 21, 'total_tokens': 45}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-2fa938ae-89a6-4535-ba07-5a2a938d57c4-0', usage_metadata={'input_tokens': 21, 'output_tokens': 24, 'total_tokens': 45})]

In [22]:
combined.batch([{"country" : "대한민국"}, {"country" : "미국"}])

[{'capital': AIMessage(content='대한민국의 수도는 서울이야.', response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 22, 'total_tokens': 38}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c5bc5fb5-7884-4064-b4f7-ae6d625babfc-0', usage_metadata={'input_tokens': 22, 'output_tokens': 16, 'total_tokens': 38}),
  'area': AIMessage(content='대한민국의 총 면적은 약 100,363 km²입니다.', response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 24, 'total_tokens': 48}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-2e456557-a38a-4cbd-b136-cfc347dd8d1d-0', usage_metadata={'input_tokens': 24, 'output_tokens': 24, 'total_tokens': 48})},
 {'capital': AIMessage(content='미국의 수도는 워싱턴 D.C.입니다.', response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 19, 'total_tokens': 36}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finis